In [1]:
#pip install --upgrade -q openai langchain langchain-openai langchain-community langgraph

In [109]:
import os
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, tool
from typing import List
import requests
import json
import os
import select
from datetime import datetime
from textwrap import dedent
from typing import Dict, List, Optional, TypeVar, Union
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic import ValidationError
from typing import Annotated, Any, Dict, List, Literal, Optional, TypedDict, TypeVar, Union
from IPython.display import Image, display
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.runnables import Runnable, RunnableConfig, RunnableLambda
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, START, StateGraph
from langgraph.graph import MessagesState
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
import sys
import uuid
from pprint import pprint

In [110]:
load_dotenv()

True

In [111]:
OPENAI_MODEL = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [112]:
def login():
    USUARIO = os.getenv("USUARIO")
    SENHA = os.getenv("SENHA")
    APP_VERSION = os.getenv("APP_VERSION")
    PARTNER_ID = os.getenv("PARTNER_ID")
    URL_BASE = os.getenv("URL_BASE")

    url = f"{URL_BASE}/ecommerce/authentication"
    
    payload = json.dumps({
        "username": USUARIO,
        "password": SENHA
    })

    headers = {
        "appversion": f"{APP_VERSION}",
        "Content-Type": "application/json",
    }

    try:
        response = requests.post(url, data=payload, headers=headers)
        response.raise_for_status()  
        return response.json().get("token")
    except Exception as e:
        msg = f"Erro ao autenticar: {str(e)}"
        raise Exception(msg)

In [113]:
token = login()
print("Login OK!!!")

Login OK!!!


In [114]:
def pesquisar_produtos(lista_produtos: List[str]) -> List[dict]:
    USUARIO = os.getenv("USUARIO")
    SENHA = os.getenv("SENHA")
    APP_VERSION = os.getenv("APP_VERSION")
    PARTNER_ID = os.getenv("PARTNER_ID")
    URL_BASE = os.getenv("URL_BASE")

    token = login()
    
    url_base = f"{URL_BASE}/ecommerce/purchase/search/_produto_?page=0&size={5}"
    product_list = []

    for produto in lista_produtos:
        produto = produto.replace(' ', '%20')
        url = url_base.replace("_produto_", produto)

        headers = {
            'appversion': f'{APP_VERSION}',
            'handlerpartner': f'{PARTNER_ID}',
            'Authorization': f'Bearer {token}'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()


In [115]:
print(pesquisar_produtos(["sobrecoxa"]))

{'page': {'page': 0, 'items': [{'product': {'id': '790931cf-c732-4f16-9e3a-066e5c0e61e0', 'code': '97210', 'skuEcommerce': None, 'family': {'id': '0ae5dae8-a3b6-4d8a-b64f-dceb85a16a85', 'type': 'FAMILY', 'name': 'COXA COM SOBRECOXA DE FRANGO PACOTE +/- 1KG CONGELADO'}, 'brand': {'id': '88067550-ff12-48a0-8a71-2439a47b9f8d', 'type': 'BRAND', 'name': 'CANCAO'}, 'validity': '2025-12-09', 'price': {'salesPrice': 9.34, 'price': 9.34, 'promotions': []}, 'available': None, 'deleted': False, 'medias': ['https://segalas-ecommerce-medias.s3.sa-east-1.amazonaws.com/produtos/97210.jpg'], 'characteristics': ['CONGELADO', 'INDIVIDUAL'], 'packages': [{'id': '5a7a1473-f5d7-4dff-a6df-f140d5a3b784', 'description': 'CX 18KG', 'label': '18x1Kg', 'weight': 18.0, 'unities': 18.0, 'salesUnitMultiplier': 18.0, 'packageType': 'BOX', 'deleted': False}], 'salesUnit': 'KG', 'isFefo': False}, 'canDelete': None, 'hasHistoric': True, 'favorite': False, 'lastOrder': {'date': '2024-11-25', 'packaging': 'CX 18KG', 'qua

In [116]:
@tool
def tool_pesquisa_produto(lista_produtos: List[str]):
    """
    Útil para pesquisar informações, preços ou disponibilidade de produtos.

    Args:
    - lista_produtos: Lista de produtos para pesquisar. Ex: ["batata doce", "miolo de alcatra"]

    Exemplos:
    - qual o preço do miolo de alcatra e da batata doce?
    - quanto custa a picanha e a costela bovina?
    """
    print("\n***\n")
    print(lista_produtos)
    print("\n***\n")
    resultado = pesquisar_produtos(lista_produtos)
    print("\n***\n")
    return resultado
    print("\n***\n")

In [117]:
pergunta = "qual o preço do lombinho e da lasanha de presunto e queijo?"

In [118]:
tools = []
tools.append(tool_pesquisa_produto)
tool_node = ToolNode(tools)

In [119]:
llm = ChatOpenAI(model=OPENAI_MODEL,
                 api_key=OPENAI_API_KEY,
                 temperature=0.7,
                 verbose=True)

In [120]:
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [121]:
prompt_str = dedent("""
Responda a pergunta abaixo APENAS com base nas ferramentas e no contexto fornecido.
Em hipótese alguma consulte sua base de conhecimento.
Se você não tiver dados suficientes no contexto, diga: não sei responder.

Solicitação do cliente:
{input}
""")

In [122]:
assistant_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt_str),
            ("human", "{input}"),
            ("placeholder", "{messages}"),
        ]
    )
model = assistant_prompt | llm_with_tools

In [123]:
contexto = {
    "messages": [HumanMessage(content=pergunta)],
    "input": pergunta
}

In [124]:
class Assistant:
    def __init__(self, runnable: Runnable):
        # Initialize with the runnable that defines the process for interacting with the tools
        self.runnable = runnable

    def __call__(self, state: MessagesState):
        while True:
            messages = state["messages"]
            last_message = messages[-1]
            result = self.runnable.invoke(messages)
            
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": [result]}

In [126]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", Assistant(model))
builder.add_node("tools", tool_node)
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)

In [127]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [128]:
config = {"configurable": {"thread_id": "1"}}

In [129]:
result = await graph.ainvoke(contexto, config, debug=True)
answer = result["messages"][-1].content
print("\n\n***\n\n")
print(result)
print(answer)
print("\n\n***\n\n")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'input': 'qual o preço do lombinho e da lasanha de presunto e queijo?',
 'messages': [HumanMessage(content='qual o preço do lombinho e da lasanha de presunto e queijo?', additional_kwargs={}, response_metadata={})]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='qual o preço do lombinho e da lasanha de presunto e queijo?', additional_kwargs={}, response_metadata={})]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='qual o preço do lombinho e da lasanha de presunto e queijo?', additional_kwargs={}, response_metadata={}, id='9d83bdbf-744a-47b0-94f0-dd0bd52eb643')]}
[1:tasks] Starting 1 task for step 1:
- assistant -> {'messages': [HumanMessage(content='qual o preço do lombinho e da lasanha de presunto e queijo?', additional_kwargs={}, response_metadata={}, id='9d83bdbf-744a-47b0-94f0-dd0bd52eb643')]}
[